In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy

In [2]:
mov = pd.read_csv("Resources/movie_metadata.csv")
# mov = read_sql('movie_db.sqlite')
# print(mov.columns.values)
# print(mov.head())
mov_df = mov
mov_df.columns.values

array(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes',
       'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres',
       'actor_1_name', 'movie_title', 'num_voted_users',
       'cast_total_facebook_likes', 'actor_3_name',
       'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link',
       'num_user_for_reviews', 'language', 'country', 'content_rating',
       'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score',
       'aspect_ratio', 'movie_facebook_likes'], dtype=object)

In [3]:
# Unique directoe count
mov_df.director_name.nunique()

2398

In [4]:
# Unique actor count
mov_df.groupby(['actor_1_name', 'actor_2_name', 'actor_2_name']).ngroups
len(mov_df)

5043

In [5]:
mov_df.drop_duplicates(inplace=True)
len(mov_df)

4998

In [6]:
# Find all null values
mov_df.isnull().sum()

color                         19
director_name                103
num_critic_for_reviews        49
duration                      15
director_facebook_likes      103
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        874
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                152
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               301
budget                       487
title_year                   107
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 327
movie_facebook_likes           0
dtype: int64

In [7]:
# Since gross and budget have too many missing values, and we want to keep these two variables for the following analysis, we can only delete rows with null values for gross and budget because imputation will not do a good job here.

mov_df = mov_df.dropna(subset=['gross', 'budget'])
len(mov_df)

3857

In [8]:
# let’s take a look at rest columns with missing values.
mov_df.isnull().sum()

color                         2
director_name                 0
num_critic_for_reviews        1
duration                      1
director_facebook_likes       0
actor_3_facebook_likes       10
actor_2_name                  5
actor_1_facebook_likes        3
gross                         0
genres                        0
actor_1_name                  3
movie_title                   0
num_voted_users               0
cast_total_facebook_likes     0
actor_3_name                 10
facenumber_in_poster          6
plot_keywords                31
movie_imdb_link               0
num_user_for_reviews          0
language                      3
country                       0
content_rating               51
budget                        0
title_year                    0
actor_2_facebook_likes        5
imdb_score                    0
aspect_ratio                 74
movie_facebook_likes          0
dtype: int64

In [9]:
# aspect_ratio has the highest number of missing values. 
# Before trying to impute the missing values, let's check how important is this variable.
pd.value_counts(mov_df.aspect_ratio)

2.35     1995
1.85     1600
1.37       50
1.78       41
1.66       40
1.33       19
2.39       11
2.20       10
2.40        3
2.76        3
2.00        3
1.75        2
2.24        1
1.18        1
2.55        1
1.77        1
16.00       1
1.50        1
Name: aspect_ratio, dtype: int64

In [10]:
# mov_df['aspect_ratio']

In [11]:
# The most common aspect ratios are 1.85 and 2.35. For analyzing purpose, we group other ratios together.
# In order to compute the mean of imdb score for different aspect_ratio, we need to replace NA with 0 first.
mov_df.loc[:,('aspect_ratio')].fillna(0, inplace=True)

ar_235 = mov_df["aspect_ratio"] == 2.35
ar_185 = mov_df["aspect_ratio"] == 1.85
ar_others = (mov_df["aspect_ratio"] != 2.35) & (mov_df["aspect_ratio"] != 1.85)
# mov_df["aspect_ratio"].loc[].mean()
ar_235_mean = mov_df.loc[ar_235, 'imdb_score'].mean()
ar_185_mean = mov_df.loc[ar_185, 'imdb_score'].mean()
ar_others_mean = mov_df.loc[ar_others, 'imdb_score'].mean()
print(ar_235_mean)
print(ar_185_mean)
print(ar_others_mean)


6.508471177944862
6.373937500000018
6.672519083969459


/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
# From the means of imdb score for different aspect ratios, we can see there is no significant difference, 
# all the means fall in the range of 6.3~6.8. So, removing this variable won’t affect our following analysis.

mov_df.drop(['aspect_ratio'], axis=1, inplace=True)
mov_df.columns.values

/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes',
       'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres',
       'actor_1_name', 'movie_title', 'num_voted_users',
       'cast_total_facebook_likes', 'actor_3_name',
       'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link',
       'num_user_for_reviews', 'language', 'country', 'content_rating',
       'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score',
       'movie_facebook_likes'], dtype=object)

In [13]:
mov_df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,http://www.imdb.com/title/tt0398286/?ref_=fn_t...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,http://www.imdb.com/title/tt2395427/?ref_=fn_t...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,http://www.imdb.com/title/tt0417741/?ref_=fn_t...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,10000
10,Color,Zack Snyder,673.0,183.0,0.0,2000.0,Lauren Cohan,15000.0,330249062.0,Action|Adventure|Sci-Fi,...,http://www.imdb.com/title/tt2975590/?ref_=fn_t...,3018.0,English,USA,PG-13,250000000.0,2016.0,4000.0,6.9,197000


In [14]:
# We notice that there are some 0 values which should also be regarded as missing value except for predictor facenumber_in_poster.
# First we need to replace NA with column average for facenumber_in_poster, 

face_number_mean = mov_df['facenumber_in_poster'].mean()
print(face_number_mean)
mov_df['facenumber_in_poster'].fillna(face_number_mean, inplace=True)


1.376785250584264


/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
# then replace 0s in other predictors with NA, 
# cols = mov_df.columns.values
cols = mov_df.loc[:, mov_df.columns != 'facenumber_in_poster']
cols = cols.columns.values.tolist()
print(cols)
mov_df[cols] = mov_df[cols].replace({'0':np.nan, 0:np.nan})


['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'movie_facebook_likes']


/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [16]:


# and lastly replace all NAs with their respective column mean.
# print(mov_df.mean())
mov_df = mov_df.fillna(mov_df.mean(), inplace=True)

/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
mov_df.dropna(subset=['content_rating'], inplace=True)
mov_df = mov_df[mov_df.content_rating != ""]


/Users/Shameetha/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
# Replace M and GP with PG, replace X with NC-17, because these two are what we use nowadays.
# Replace “Approved”, “Not Rated”, “Passed”, “Unrated” with the most common rating “R”.
mov_df['content_rating'] = mov_df['content_rating'].replace({
                                                            'M':'PG', 
                                                            'GP':'PG', 
                                                            'X' : 'NC-17',
                                                            'Approved' : 'R',
                                                            'Not Rated' : 'R',
                                                            'Passed' : 'R',
                                                            'Unrated' : 'R',
                                                            })

In [19]:
pd.value_counts(mov_df.color)

Color               3680
 Black and White     124
Name: color, dtype: int64

In [20]:
# delete predictor color, movie link, title
# mov_df.drop(['color'], axis=1, inplace=True)
# mov_df.drop(['movie_imdb_link'], axis=1, inplace=True)
# mov_df.drop(['movie_title'], axis=1, inplace=True)

In [21]:
len(mov_df)

3806

In [22]:
mov_df.to_csv('Resources/cleaned_data.csv')